## 1.1 - Basic LLM Prompting with vLLM, Langchain and Prompt with Conversation Memory

## Preparation

In [1]:
!pip install -q langgraph==0.2.35 langchain_experimental==0.0.65 langchain-openai==0.1.25 termcolor==2.3.0 duckduckgo_search==7.1.0 openapi-python-client==0.12.3 langchain_community==0.2.19 wikipedia==1.4.0

In [2]:
# Imports
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
import os

In [3]:
INFERENCE_SERVER_URL = os.getenv('API_URL')
MODEL_NAME = "mistral-7b-instruct"
API_KEY= os.getenv('API_KEY')

#### Create the LLM instance

In [4]:
# LLM definition
llm = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base= f"{INFERENCE_SERVER_URL}/v1",
    model_name=MODEL_NAME,
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

#### Create the Prompt

In [5]:
template="""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Current conversation:
{history}
Human: {input}
AI:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)

#### And finally some memory for the conversation

In [6]:
memory=ConversationBufferMemory()

## First example, fully verbose mode

#### Create the Chain using the different components

In [7]:
# Verbose mode is intentionally set to True so you can see the prompt and the history from the buffer memory
conversation = ConversationChain(llm=llm,
                                 prompt=PROMPT,
                                 verbose=True,
                                 memory=memory
                                )

/tmp/ipykernel_2184/3407751332.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  conversation = ConversationChain(llm=llm,


#### Let's talk...

In [8]:
first_input = "Describe what is the S&P500 in 100 words or less."
conversation.predict(input=first_input);

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Current conversation:

Human: Describe what is the S&P500 in 100 words or less.
AI:
[/INST]

 The S&P 500, or Standard & Poor's 500, is a stock market index that measures the performance of 500 large companies listed on stock exchanges in the United States. It is widely regarded as a primary gauge of U.S. stock market health and is often used as a benchmark for various investment strategies. Companies in the S&P 500 span multiple sectors, in

In [9]:
# This is a follow-up question without context to showcase the conversation memory
second_input = "How many companies are included?"
conversation.predict(input=second_input);

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Current conversation:
Human: Describe what is the S&P500 in 100 words or less.
AI:  The S&P 500, or Standard & Poor's 500, is a stock market index that measures the performance of 500 large companies listed on stock exchanges in the United States. It is widely regarded as a primary gauge of U.S. stock market health and is often used as a benchmark for various investment strategies. Companies in the S&P 500 span multiple sectors, including te

### Second example, no verbosity

In [10]:
# Verbose mode is intentionally set to True so you can see the prompt and the history from the buffer memory
conversation2 = ConversationChain(llm=llm,
                                 prompt=PROMPT,
                                 verbose=False,
                                 memory=memory
                                )

In [11]:
# Let's clear the previous conversation first
memory.clear()

In [12]:
first_input = "Describe what is the S&P500 results of the last 5 starting with the 2024"
conversation2.predict(input=first_input);

 I'm sorry for any confusion, but the S&P 500 results are not available for future years as they are based on historical data and have not occurred yet. The S&P 500 is an American stock market index based on the market capitalizations of 500 large companies having common stock listed on the NYSE or NASDAQ. To get the S&P 500 results for a specific year, you would typically look up the annual performance data provided by financial institutions or online resources that track stock market indices. For example, you can find the S&P 500 results for past years on websites like Yahoo Finance or Bloomberg.

In [13]:
second_input = "Are you able to perform a internet search to check the latest results?"
conversation2.predict(input=first_input);

 I apologize for any confusion, but the S&P 500 results are not available for future years as they are based on historical data and have not occurred yet. The S&P 500 is an American stock market index based on the market capitalizations of 500 large companies having common stock listed on the NYSE or NASDAQ. To get the S&P 500 results for a specific year, you would typically look up the annual performance data provided by financial institutions or online resources that track stock market indices. For example, you can find the S&P 500 results for past years on websites like Yahoo Finance or Bloomberg. However, since we are discussing the last 5 years starting from 2024, it's important to note that these results are not available as they have not been determined yet.

## What's Next?

Let's head over to give our LLM the capability of using Tools!